# 神经网络模型：

In [75]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 30

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/zj_space_1d_rate_20d_12-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

zj_c2_s0.1_b16_lr0.001_d0.5_e30
models/model_zj_c2_s0.1_b16_lr0.001_d0.5_e30.pth
True
1
GeForce MX150
0


### 构建训练集

In [76]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,0.0,600895.XSHG,MainBoard,Industrials,0,2012-01-04,0.668096,-1.603230,-1.051909,0.010194,...,-2.149004,-1.794150,-2.097427,-0.798000,-0.775429,-0.820660,0.210017,-0.349589,-1.549173,-1.406188
1,0.0,600895.XSHG,MainBoard,Industrials,1,2012-01-05,0.908286,-1.597009,-1.091188,0.058377,...,-2.187196,-1.794150,-2.235441,-0.797675,-0.774009,-0.820659,0.208850,-0.312678,-1.602482,-1.492669
2,0.0,600895.XSHG,MainBoard,Industrials,2,2012-01-06,0.988467,-1.599470,-1.075912,0.039320,...,-1.864218,-1.044691,-1.909176,-0.802157,-0.781049,-0.821995,0.173532,-0.282713,-1.718665,-1.616956
3,0.0,600895.XSHG,MainBoard,Industrials,3,2012-01-09,0.786295,-1.609752,-1.008266,-0.040334,...,-0.608269,-1.044691,-1.111682,-0.793187,-0.780847,-0.820737,0.041498,-0.264281,-1.538311,-1.347998
4,0.0,600895.XSHG,MainBoard,Industrials,4,2012-01-10,0.597582,-1.617382,-0.953712,-0.099439,...,-0.178138,-0.295232,-0.451633,-0.780389,-0.775671,-0.819612,-0.101613,-0.252861,-1.442265,-1.457513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,0.0,600895.XSHG,MainBoard,Industrials,2406,2021-11-29,0.040620,2.505919,-0.723704,-0.316174,...,-1.192598,-0.295232,-0.301873,-0.748341,-0.669583,-0.476146,-0.771154,-0.098022,-1.461414,-1.654446
2400,1.0,600895.XSHG,MainBoard,Industrials,2407,2021-11-30,-0.004882,2.473869,-0.710581,-0.326401,...,-0.555302,-0.295232,-0.191797,-0.747243,-0.674466,-0.512491,-0.764523,-0.082492,-1.568848,-1.513253
2401,1.0,600895.XSHG,MainBoard,Industrials,2408,2021-12-01,-0.029767,2.462332,-0.705808,-0.330082,...,-0.587372,0.454227,-0.144966,-0.746992,-0.676701,-0.549897,-0.763819,-0.068585,-1.444480,-1.506378
2402,0.0,600895.XSHG,MainBoard,Industrials,2409,2021-12-02,0.010390,2.491292,-0.717739,-0.320842,...,-0.316660,-0.295232,-0.224451,-0.745291,-0.692358,-0.564154,-0.757408,-0.058939,-1.300889,-1.402812


In [77]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,-1.603230,-1.051909,0.010194,-1.057518,-0.962171,-0.496158,-2.149004,-1.794150,-2.097427,-0.798000,-0.775429,-0.820660,0.210017,-0.349589,-1.549173,-1.406188
1,-1.597009,-1.091188,0.058377,-1.091523,-0.997159,-0.497766,-2.187196,-1.794150,-2.235441,-0.797675,-0.774009,-0.820659,0.208850,-0.312678,-1.602482,-1.492669
2,-1.599470,-1.075912,0.039320,-1.096094,-1.026104,-0.416796,-1.864218,-1.044691,-1.909176,-0.802157,-0.781049,-0.821995,0.173532,-0.282713,-1.718665,-1.616956
3,-1.609752,-1.008266,-0.040334,-1.043153,-1.038215,-0.191290,-0.608269,-1.044691,-1.111682,-0.793187,-0.780847,-0.820737,0.041498,-0.264281,-1.538311,-1.347998
4,-1.617382,-0.953712,-0.099439,-0.954356,-1.029381,0.089322,-0.178138,-0.295232,-0.451633,-0.780389,-0.775671,-0.819612,-0.101613,-0.252861,-1.442265,-1.457513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,0.486620,0.054286,-0.947774,-0.505349,-0.679579,0.486302,0.322967,-1.794150,-0.129465,-0.166240,-0.269208,0.852851,0.059265,-0.116384,-1.193304,-0.821015
2159,0.469203,0.073563,-0.953428,-0.445737,-0.636370,0.549833,0.414804,-1.044691,-0.046674,-0.147391,-0.276099,0.821180,0.037877,-0.176032,-1.093606,-0.641653
2160,0.503032,0.036386,-0.942447,-0.433055,-0.599157,0.469501,0.225628,-1.794150,-0.227087,-0.151157,-0.284739,0.807267,0.031992,-0.218353,-1.194527,-0.735829
2161,0.511967,0.026747,-0.939547,-0.428620,-0.568462,0.382189,0.088561,-1.794150,-0.242801,-0.184586,-0.293756,0.740537,0.004905,-0.254286,-1.175696,-0.642815


In [78]:
y_train

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2158    1.0
2159    1.0
2160    1.0
2161    1.0
2162    1.0
Name: return_bin, Length: 2163, dtype: float64

In [79]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,-1.603230,-1.051909,0.010194,-1.057518,-0.962171,-0.496158,-2.149004,-1.794150,-2.097427,-0.798000,-0.775429,-0.820660,0.210017,-0.349589,-1.549173,-1.406188,0.0
1,-1.597009,-1.091188,0.058377,-1.091523,-0.997159,-0.497766,-2.187196,-1.794150,-2.235441,-0.797675,-0.774009,-0.820659,0.208850,-0.312678,-1.602482,-1.492669,0.0
2,-1.599470,-1.075912,0.039320,-1.096094,-1.026104,-0.416796,-1.864218,-1.044691,-1.909176,-0.802157,-0.781049,-0.821995,0.173532,-0.282713,-1.718665,-1.616956,0.0
3,-1.609752,-1.008266,-0.040334,-1.043153,-1.038215,-0.191290,-0.608269,-1.044691,-1.111682,-0.793187,-0.780847,-0.820737,0.041498,-0.264281,-1.538311,-1.347998,0.0
4,-1.617382,-0.953712,-0.099439,-0.954356,-1.029381,0.089322,-0.178138,-0.295232,-0.451633,-0.780389,-0.775671,-0.819612,-0.101613,-0.252861,-1.442265,-1.457513,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,0.486620,0.054286,-0.947774,-0.505349,-0.679579,0.486302,0.322967,-1.794150,-0.129465,-0.166240,-0.269208,0.852851,0.059265,-0.116384,-1.193304,-0.821015,1.0
2159,0.469203,0.073563,-0.953428,-0.445737,-0.636370,0.549833,0.414804,-1.044691,-0.046674,-0.147391,-0.276099,0.821180,0.037877,-0.176032,-1.093606,-0.641653,1.0
2160,0.503032,0.036386,-0.942447,-0.433055,-0.599157,0.469501,0.225628,-1.794150,-0.227087,-0.151157,-0.284739,0.807267,0.031992,-0.218353,-1.194527,-0.735829,1.0
2161,0.511967,0.026747,-0.939547,-0.428620,-0.568462,0.382189,0.088561,-1.794150,-0.242801,-0.184586,-0.293756,0.740537,0.004905,-0.254286,-1.175696,-0.642815,1.0


In [80]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [81]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [82]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [83]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [84]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [85]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [86]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [87]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [88]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [89]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [90]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 54.2%, Avg loss: 0.691567 

Precision of every train dataset class:  tensor([0.3852, 0.5751], device='cuda:0')

Test Error: 
    Accuracy: 69.3%, Avg loss: 0.651040 

Precision of every test dataset class:  tensor([0.0000, 0.6929], device='cuda:0')

Time cost = 2.387602s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 58.3%, Avg loss: 0.681130 

Precision of every train dataset class:  tensor([0.5132, 0.5855], device='cuda:0')

Test Error: 
    Accuracy: 69.3%, Avg loss: 0.641971 

Precision of every test dataset class:  tensor([0.0000, 0.6929], device='cuda:0')

Time cost = 4.102203s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 58.6%, Avg loss: 0.677093 

Precision of every train dataset class:  tensor([0.5789, 0.5864], device='cuda:0')

Test Error: 
    Accuracy: 69.3%, Avg loss: 0.653161 

Precision of every test dataset class:  tensor([0.0000, 0.6929], device='cuda:0')



## 保存模型

In [91]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [92]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [93]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [94]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [95]:
# X_cv.columns.values.tolist()

In [96]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [97]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [98]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)